## The purpose of this is to begin incorporating Clare Smith's additional TnSeq screens: 

In [1]:
import pandas as pd
import os

In [4]:
dir_data = '../../data/standardized_data/'
file_logfc = os.path.join(dir_data, 'result_logfc_matrix_2021_05_18.csv')
file_qvals = os.path.join(dir_data, 'result_pval_matrix_2021_05_18.csv')

df_logfc = pd.read_csv(file_logfc)
df_qvals = pd.read_csv(file_qvals)

df_logfc.rename(columns={df_logfc.columns[0]:'Rv_ID'}, inplace=True)
df_qvals.rename(columns={df_qvals.columns[0]:'Rv_ID'}, inplace=True)


In [5]:
cols_to_drop = [col for col in df_qvals.columns if "carey" in col and "mbio_H37Rv" in col]

df_qvals.drop(cols_to_drop, axis = 1, inplace = True)
df_logfc.drop(cols_to_drop, axis = 1, inplace = True)

## Which columns overlap with the older datasets?

In [9]:
file_logfc_OLD = os.path.join(dir_data, 'result_logfc_matrix_2020_06_27.csv')
file_qvals_OLD = os.path.join(dir_data, 'result_qval_matrix_2020_06_27.csv')

df_logfc_OLD = pd.read_csv(file_logfc_OLD)
df_qvals_OLD = pd.read_csv(file_qvals_OLD)

df_qvals_OLD.drop(cols_to_drop, axis = 1, inplace = True)
df_logfc_OLD.drop(cols_to_drop, axis = 1, inplace = True)

In [10]:
cols_OLD = [col for col in df_logfc.columns if col in df_logfc_OLD.columns]
cols_NEW = [col for col in df_logfc.columns if not col in df_logfc_OLD.columns]

In [11]:
print(len(cols_OLD), len(cols_NEW))

65 1897


# Binarize: 

In [12]:
# set thresholds
qval_thresh = 0.05
log2fc_tresh = 1
# initialized binary dataframe
df_bin = df_qvals.copy()

cols_data = df_qvals.columns[1:]
for col in cols_data:
    df_bin.loc[ (df_qvals[col] <= qval_thresh) & (df_logfc[col].abs() >= log2fc_tresh), col ] = 1
    df_bin.loc[ (df_qvals[col] > qval_thresh), col ] = 0  
    df_bin.loc[(df_logfc[col].abs() < log2fc_tresh), col] = 0

In [13]:
df_bin.shape

(4055, 1962)

Write to file: 

In [50]:
file_qvals_out = os.path.join(dir_data, 'result_qval_matrix_2021_08_27.csv')
file_log2fc_out = os.path.join(dir_data, 'result_logfc_matrix_2021_08_27.csv')
file_bin_out = os.path.join(dir_data, 'result_bin_matrix_2021_08_27.csv')

df_bin.to_csv(file_bin_out, index = False)
df_qvals.to_csv(file_qvals_out, index = False)
df_logfc.to_csv(file_log2fc_out, index = False)

## Detect the "basis set" columns for Clare's dataset: 

In [14]:
cols_ref_mouse = [col for col in cols_NEW if 'C57BL' in col]
len(cols_ref_mouse)

64

In [21]:
df_logfc_BASIS = df_logfc[cols_OLD + cols_ref_mouse]
df_qvals_BASIS = df_qvals[cols_OLD + cols_ref_mouse]
df_bin_BASIS = df_bin[cols_OLD + cols_ref_mouse]

In [22]:
df_bin_BASIS.shape

(4055, 129)

In [23]:
file_qvals_out = os.path.join(dir_data, 'result_qval_matrix_2021_08_27_BASIS.csv')
file_log2fc_out = os.path.join(dir_data, 'result_logfc_matrix_2021_08_27_BASIS.csv')
file_bin_out = os.path.join(dir_data, 'result_bin_matrix_2021_08_27_BASIS.csv')

df_bin_BASIS.to_csv(file_bin_out, index = False)
df_qvals_BASIS.to_csv(file_qvals_out, index = False)
df_logfc_BASIS.to_csv(file_log2fc_out, index = False)